In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/transformers/2b/2/model.safetensors.index.json
/kaggle/input/gemma/transformers/2b/2/gemma-2b.gguf
/kaggle/input/gemma/transformers/2b/2/config.json
/kaggle/input/gemma/transformers/2b/2/model-00001-of-00002.safetensors
/kaggle/input/gemma/transformers/2b/2/model-00002-of-00002.safetensors
/kaggle/input/gemma/transformers/2b/2/tokenizer.json
/kaggle/input/gemma/transformers/2b/2/tokenizer_config.json
/kaggle/input/gemma/transformers/2b/2/special_tokens_map.json
/kaggle/input/gemma/transformers/2b/2/.gitattributes
/kaggle/input/gemma/transformers/2b/2/tokenizer.model
/kaggle/input/gemma/transformers/2b/2/generation_config.json


In [3]:
import pandas as pd

df = pd.read_json("hf://datasets/giuliadc/cnndm_5k/cnndm_5k_2-0.630-test.json")

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel, GemmaForCausalLM 
import torch
# Load the tokenizer and the correct model
model_name = '/kaggle/input/gemma/transformers/2b/2'  # Update with your model path
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GemmaForCausalLM.from_pretrained(model_name)  # Use the appropriate model class
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)
model.to(device)  

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [5]:
df.head()

,id,text,reference-summary,source
0,cnndm-1,A flickering rainbow of colours came with the ...,"Italian photographer Giovanna Griffo, 42, shar...",4ab4bb5edcb0a9b97b87a0c4a9a721984210c2f4
1,cnndm-2,Ed Sheeran surprised a fan in hospital for her...,"Jess Knight, 20, was surprised by Ed Sheeran a...",ac1b0180881c97d9f0d57d403c0bbc21fe29dd3c
2,cnndm-3,It’s been one of the most indulgent shopping s...,"Lily James, the British star of the upcoming C...",8f3a478ae49dfe072a204b60c2577c2375341628
3,cnndm-4,The Block’s up market South Yarra apartments a...,The Block’s four apartments are now on the mar...,3f87b31923a09bf2248f7b3bc3303bb0c9ebdbe4
4,cnndm-5,"(CNN)Recently, a New York judge issued an opin...",A court allowed a wife to serve divorce papers...,8610ead0a43054be202a1fc756620415a6d572fe


In [6]:
!pip install bert-score
!pip install rouge-score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import nltk
from transformers.modeling_utils import prune_linear_layer
from collections import defaultdict


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d2f84b2fc77bf311f19b409d6177d947a3f736a3141ba7b8a821204ff026f275
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [7]:
# Download required packages for BLEU
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
def summarize_text(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=1024, truncation=True).to("cuda")
    # Use max_new_tokens to control the output length while keeping input handling flexible
    summary_ids = model.generate(
        inputs.input_ids, 
        max_new_tokens=150,  # Generates up to 150 tokens in the summary
        min_length=50, 
        length_penalty=2.0, 
        num_beams=4, 
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [9]:
from transformers import AutoTokenizer, GemmaForCausalLM
# Take the first 100 rows

df_100 = df.head(30)
# df_100.to(device)
# Apply the summarization to the 'text' column for the first 100 rows
df_100['generated_summary'] = df_100['text'].apply(summarize_text)


/tmp/ipykernel_30/837514614.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['generated_summary'] = df_100['text'].apply(summarize_text)


In [10]:
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to compute BLEU and ROUGE scores
def compute_scores(reference, hypothesis):
    # BLEU score
    bleu_score = sentence_bleu([reference.split()], hypothesis.split())
    
    # ROUGE score
    rouge_scores = scorer.score(reference, hypothesis)
    
    return bleu_score, rouge_scores

# List to store scores
results = []

# Iterate through the first 100 rows to compute BLEU and ROUGE scores
for i, row in df_100.iterrows():
    reference = row['reference-summary']
    hypothesis = row['generated_summary']
    
    # Calculate BLEU and ROUGE scores
    bleu_score, rouge_scores = compute_scores(reference, hypothesis)
    
    # Store results
    results.append({
        'id': row['id'],
        'bleu_score': bleu_score,
        'rouge1': rouge_scores['rouge1'].fmeasure,
        'rouge2': rouge_scores['rouge2'].fmeasure,
        'rougeL': rouge_scores['rougeL'].fmeasure
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display results
print(results_df.head())

# # Save results to a CSV file
# results_df.to_csv('/path_to/evaluation_results.csv', index=False)

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


        id  bleu_score    rouge1    rouge2    rougeL
0  cnndm-1    0.012572  0.093126  0.044444  0.082040
1  cnndm-2    0.024412  0.154278  0.067511  0.109397
2  cnndm-3    0.012623  0.113350  0.045455  0.080605
3  cnndm-4    0.019106  0.147087  0.065072  0.093601
4  cnndm-5    0.067139  0.032020  0.007407  0.022167


In [11]:
from bert_score import score

# Assuming df_100 has 'generated_summary' and 'reference-summary' columns
generated_summaries = df_100['generated_summary'].tolist()
reference_summaries = df_100['reference-summary'].tolist()

# Calculate BERTScore
P, R, F1 = score(generated_summaries, reference_summaries, lang='en', verbose=True)

# Output the scores
print(f'Precision: {P.mean().item():.4f}')
print(f'Recall: {R.mean().item():.4f}')
print(f'F1 Score: {F1.mean().item():.4f}')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 3.29 seconds, 9.11 sentences/sec
Precision: 0.8032
Recall: 0.8843
F1 Score: 0.8418


In [12]:
import random
def create_random_binary_list(length, percentage_of_zeros):
    num_zeros = int(length * percentage_of_zeros)
    num_ones = length - num_zeros

    # Create the list with the required number of 0s and 1s
    binary_list = [0] * num_zeros + [1] * num_ones

    # Shuffle the list to randomize the order
    random.shuffle(binary_list)

    return binary_list

def initialize_chromosome(num_genes):
  # initialize chromosome with given sparsity percentage
  return create_random_binary_list(num_genes, SPARSITY_RATE)

In [13]:
def make_model():
    # del model
    model = GemmaForCausalLM.from_pretrained(model_name)  # Use the appropriate model class
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = torch.nn.DataParallel(model)
    return model

In [22]:
def modify_model(model, chromosome):
    num_heads = model.config.num_attention_heads
    num_blocks = model.config.num_hidden_layers
    # num_ffn = model.config.num_hidden_layers

    # Disable attention heads
    heads_to_prune = defaultdict(list)
    for i, gene in enumerate(chromosome):
        if gene == 0:
          block_num = i//num_heads
          head_num = i%num_heads
          heads_to_prune[block_num].append(head_num)

    head_dim = model.config.head_dim
    if heads_to_prune:
        for block_num in heads_to_prune:
            keep_indices = []
            for head in range(num_heads):
                # Get the range of indices for this head
                start_idx = head * head_dim
                end_idx = (head + 1) * head_dim
                
                if head not in heads_to_prune:
                    keep_indices += list(range(start_idx, end_idx))
            
            # Convert the keep indices into a LongTensor
            keep_indices_tensor = torch.LongTensor(keep_indices)

            model.model.layers[block_num].self_attn.q_proj = prune_linear_layer(model.model.layers[block_num].self_attn.q_proj, keep_indices_tensor)
            model.model.layers[block_num].self_attn.k_proj = prune_linear_layer(model.model.layers[block_num].self_attn.k_proj, keep_indices_tensor)
            model.model.layers[block_num].self_attn.v_proj = prune_linear_layer(model.model.layers[block_num].self_attn.v_proj, keep_indices_tensor)
            model.model.layers[block_num].self_attn.o_proj = prune_linear_layer(model.model.layers[block_num].self_attn.o_proj, keep_indices_tensor)
    return model

In [15]:
def evaluate_fitness(chromosome, dataset):
    del model
    
    df_100['generated_summary'] = dataset['text'].apply(summarize_text)
    df_100['reference-summary'] = dataset['reference-summary']
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # List to store scores
    results = []
    
    # Iterate through the first 100 rows to compute BLEU and ROUGE scores
    for i, row in df_100.iterrows():
        reference = row['reference-summary']
        hypothesis = row['generated_summary']
        
        # Calculate BLEU and ROUGE scores
        bleu_score, rouge_scores = compute_scores(reference, hypothesis)
        
        # Store results
        results.append({
            'id': row['id'],
            'bleu_score': bleu_score,
            'rouge1': rouge_scores['rouge1'].fmeasure,
            'rouge2': rouge_scores['rouge2'].fmeasure,
            'rougeL': rouge_scores['rougeL'].fmeasure
        })
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    generated_summaries = df_100['generated_summary'].tolist()
    reference_summaries = df_100['reference-summary'].tolist()

    # Calculate BERTScore
    P, R, F1 = score(generated_summaries, reference_summaries, lang='en', verbose=True)
    return P.mean().item()


def compute_scores(reference, hypothesis):
    # BLEU score
    bleu_score = sentence_bleu([reference.split()], hypothesis.split())
    
    # ROUGE score
    rouge_scores = scorer.score(reference, hypothesis)
    
    return bleu_score, rouge_scores

In [16]:
SPARSITY_RATE = 0.3

In [17]:
def find_size(model):
  total_size_in_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
  total_size_in_megabytes = total_size_in_bytes / (1024 ** 2)
  print(f"Model size: {total_size_in_megabytes:.2f} MB")

In [18]:
del model
model = make_model()
find_size(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model size: 9560.29 MB


In [21]:
model.config

GemmaConfig {
  "_name_or_path": "/kaggle/input/gemma/transformers/2b/2",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 256000
}

In [23]:
chrom = initialize_chromosome(model.config.num_attention_heads*model.config.num_hidden_layers)
model = modify_model(model, chrom)
find_size(model)

Model size: 8948.29 MB


/opt/conda/lib/python3.10/site-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [ ]:
evalua